# Sandbox to learn and play with RAW image formates

In [1]:
!pip install rawpy

## Load and visualize raw images

In [ ]:
import os
import rawpy
import numpy as np
from matplotlib import pyplot as plt
#
raw_dir = os.path.join('.', 'first-dataset', 'first-dataset-RAW')
with rawpy.imread(os.path.join(raw_dir, 'IMG_7781.CR2')) as raw:
    
    
    print(f"Image Size: {raw.sizes.raw_width} x {raw.sizes.raw_height}")
    print(f"Number of colors is {raw.num_colors}")
    print(f"Color description: {raw.color_desc}")
    print(f"Raw type: {raw.raw_type}")
    # Read RAW Sensor Data, _visable crops away borders.
    raw_data = raw.raw_image_visible  # Extract sensor data (Bayer pattern)

    # Normalize data for visualization (doesn't matter for plt though, should maybe be done per channel...)
    norm_data = (raw_data - raw_data.min()) / (raw_data.max() - raw_data.min())

    fig, ax = plt.subplots(1,2, width_ratios=(0.7,0.3),figsize=(10,10))
    # Display RAW Bayer Data
    ax[0].imshow(raw_data, cmap='gray')
    ax[0].set_title("Bayer Pattern (Grayscale Raw Data)")
    # Convert to RGB using Demosaicing
    rgb_image = raw.postprocess()
    # Display Processed RGB Image
    ax[1].imshow(rgb_image)
    ax[1].set_title("Processed RGB Image")
    plt.show()

### Zoom in on sky to see if we can notice the Bayer pattern

We see on the generated image a substantally lower value in the reds, this is corresponding to the cyan-ish blue of the sky and building.
I admit I am suprised by the green being brighter than the blues though.

In [ ]:
def get_color(raw, y, x):
    return raw.color_desc.decode('utf-8')[raw.raw_color(y,x)]

with rawpy.imread(os.path.join(raw_dir, 'IMG_7781.CR2')) as raw:


    # Define the region to zoom in (e.g., top-left corner)
    w = 50
    y = 700
    x = 3250
    zoom_region = raw.raw_image_visible[y:y+w, x:x+w]

    # # Normalize the zoomed region for visualization
    # norm_zoom_region = (zoom_region - zoom_region.min()) / (zoom_region.max() - zoom_region.min())

    # Plot the zoomed region
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    im = ax.imshow(zoom_region, cmap='gray')
    fig.colorbar(im, ax = ax)
    ax.set_title("Zoomed Bayer Pattern (Grayscale Raw Data)")
    label_pixel_colors = False
    if label_pixel_colors:
        for i in range(w):
            for j in range(w):
                ax.text(j, i, get_color(raw, y + i, x + j), ha='center', va='center', fontsize=4, color='red')

    plt.show()
    print()

## Extract each channel individually.


In [53]:
def pack_raw(raw):
    # pack Bayer image to 4 channels
    im = raw.raw_image_visible.astype(np.float32)

    im = (im - im.min()) / (im.max() - im.min()) 
    im = np.expand_dims(im, axis=2)
    img_shape = im.shape
    H = img_shape[0]
    W = img_shape[1]

    out = np.concatenate((im[0:H:2, 0:W:2, :],
                          im[0:H:2, 1:W:2, :],
                          im[1:H:2, 1:W:2, :],
                          im[1:H:2, 0:W:2, :]), axis=2)
    return out

In [ ]:
with rawpy.imread(os.path.join(raw_dir, 'IMG_7781.CR2')) as raw1, \
     rawpy.imread(os.path.join(raw_dir, 'IMG_7782.CR2')) as raw2:

    packed1 = pack_raw(raw1)
    packed1 = np.rot90(packed1, k=1)

    packed2 = pack_raw(raw2)
    packed2 = np.rot90(packed2, k=1)

    fig, ax = plt.subplots(2,4, figsize=(15,10), sharey=True)
    for i in range(4):
        im = ax[0][i].imshow(packed1[:,:,i], cmap='gray')
        ax[0][i].set_title(raw1.color_desc.decode('utf-8')[i])
        if i == 0:
            fig.colorbar(im, ax = ax.ravel())
    for i in range(4):
        im = ax[1][i].imshow(packed2[:,:,i], cmap='gray')
            
    plt.show()
    print()